In [1]:
import os
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Configuración de memoria GPU debe estar antes de inicializar TensorFlow
import tensorflow as tf

In [2]:
# Configuración de parámetros
class configuracion_parametros:
    train_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
    test_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')
    sample_submission_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv')
    
    model_input_path = "/kaggle/input/neural-network-with-nash-equilibrium/keras/default/1/saved_model.keras"
    model_output_path = "/kaggle/working/saved_model.keras"  
    
    dnn_title = 'Neural Network with Nash Equilibrium'
    
    batch_size = 233234 #60000
    early_stop = 500
    n_splits = 5
    color = '#C9A9A6'
    dnn_epochs = 200
    dnn_batch_size = 64
    learning_rate = 0.0001  # Ajuste para mejor convergencia

In [3]:
# Configurar crecimiento de memoria antes de inicializar cualquier contexto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs configuradas con crecimiento de memoria: {gpus}")
    except RuntimeError as e:
        print(f"Error al configurar GPUs: {e}")
else:
    print("No se encontraron GPUs.")

GPUs configuradas con crecimiento de memoria: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
# Continuar con las importaciones restantes
import sys
import numpy as np
import polars as pl
import pandas as pd
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder, StandardScaler
import kaggle_evaluation.mcts_inference_server
from math import sqrt

In [5]:
# Usar MirroredStrategy después de configurar las GPUs
strategy = tf.distribute.MirroredStrategy()

In [6]:
# Callback para el aprendizaje
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [7]:
# Definir custom_loss y callbacks después de las importaciones
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred)) + 0.01 * tf.reduce_sum(tf.abs(y_pred))

In [8]:
# Preprocesamiento
class procesamiento:
    
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
    def drop_cols(self, df, bad_cols):
        df = df.drop([col for col in bad_cols if col in df.columns])
        df = df.drop([col for col in df.columns if df.select(pl.col(col).null_count()).item() == df.height])                
        return df
    
    def cast_datatypes(self, df):
        # Primero identificamos las columnas categóricas y las dejamos intactas
        cat_cols = df.select(pl.col(pl.String)).columns
        df = df.with_columns([pl.col(col).cast(pl.Categorical) for col in cat_cols])  # Convertir a Categorical

        # Convertimos el resto de las columnas a tipos apropiados (Int o Float)
        for col in df.columns:
            if col not in cat_cols:
                try:
                    val = df.select(pl.col(col).drop_nulls().first()).item()
                    if isinstance(val, int):
                        df = df.with_columns(pl.col(col).cast(pl.Int16))  # Convierte a Int16 si es un entero
                    else:
                        df = df.with_columns(pl.col(col).cast(pl.Float32))  # Convierte a Float32 si no es un entero
                except Exception as e:
                    print(f"Error en la conversión de columna {col}: {e}")
                    continue
        return df
    
    def apply_prosecution_data(self, path, bad_cols):
        df = pl.read_csv(path, batch_size=self.batch_size)        
        df = self.drop_cols(df, bad_cols)        
        df = self.cast_datatypes(df)
        cat_cols = [col for col in df.columns if df[col].dtype == pl.Categorical]  # Identificar las columnas categóricas
        return df, cat_cols

    def apply_prosecution_data_from_polars_df(self, df, bad_cols):
        df = self.drop_cols(df, bad_cols)        
        df = self.cast_datatypes(df)
        cat_cols = [col for col in df.columns if df[col].dtype == pl.Categorical]  # Identificar las columnas categóricas
        return df, cat_cols

prosecution_data = procesamiento(configuracion_parametros.batch_size)

In [9]:
# Función para codificar columnas categóricas
def encode_categorical_columns(data, cat_cols):
    for col in cat_cols:
        if col in data.columns:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
    return data

def get_x(x: pl.DataFrame, bad_cols):
    X, cat_cols = prosecution_data.apply_prosecution_data_from_polars_df(x, bad_cols)        
    X = X.to_pandas()
        
    # Codificar solo las columnas categóricas relevantes
    X = encode_categorical_columns(X, cat_cols)        
               
    # Escalar las características
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = X.astype(float)
    return X

def get_xy(x: pl.DataFrame, bad_cols, target):
    df, cat_cols = prosecution_data.apply_prosecution_data_from_polars_df(x, bad_cols)        
    df = df.to_pandas()

    # Obtención de las variable X y la variable objetivo (y)
    X = df.drop([target], axis=1)
    y = df[target]
    y = y.astype(float)
        
    # Codificar solo las columnas categóricas relevantes
    X = encode_categorical_columns(X, cat_cols)        
               
    # Escalar las características
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = X.astype(float)
    return X, y

In [10]:
# Modelo neuronal
class NeuralNetworkModel:
    
    def __init__(self, input_shape, output_shape, learning_rate,                  
                model_input_path,
                model_output_path):
        
        self.model_output_path = model_output_path
        self.model = None
        
        # Verificar si el archivo del modelo ya existe

        # En la cuenta
        if os.path.exists(model_input_path) and self.model == None:
            # Cargar el modelo si ya existe
            self.model = load_model(model_input_path)            
            print("Modelo cargado exitosamente desde la cuenta: ", model_input_path)

        # En el directorio de salida
        elif os.path.exists(self.model_output_path) and self.model == None:
            # Cargar el modelo si ya existe
            self.model = load_model(self.model_output_path)            
            print("Modelo cargado exitosamente desde la salida: ", self.model_output_path)

        # Si el archivo no existe, crear un modelo nuevo y guardarlo
        elif self.model == None: 
            
            print("No se encuentra el archivo del modelo. Creando un nuevo modelo en: ", self.model_output_path)

            # Definir el modelo dentro del contexto de la estrategia
            with strategy.scope():
                self.model = models.Sequential([
                                layers.Input(shape=(input_shape,)),
                                layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                                layers.Dropout(0.4),
                                layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                                layers.Dropout(0.3),
                                layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                                layers.Dense(output_shape)
                            ])
                self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                   loss='mean_squared_error', metrics=['mse'])
            # Resumen del modelo
            self.model.summary()
            
            self.model.save(self.model_output_path)  # Guardar el modelo nuevo            
            print("Nuevo modelo guardado en: ", self.model_output_path)    
    
    def fit(self, X_train, y_train, X_valid, y_valid, num_epochs, num_batch_size):
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        self.model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                          epochs=num_epochs, batch_size=num_batch_size, callbacks=[early_stop, lr_scheduler])
        self.model.save(self.model_output_path)  # Guardar el modelo después del entrenamiento
        print("Entrenamiento terminado. Modelo guardado en: ", self.model_output_path)    
    
    def predict(self, X_test):        
        return self.model.predict(X_test, verbose=0).flatten()

    def get_model(self):
        return self.model

In [11]:
# Validación del modelo
class modelo_validacion:
    
    def __init__(self, n_splits, dnn_epochs, dnn_batch_size, learning_rate, 
                 model_input_path, model_output_path):
        self.n_splits = n_splits
        self.dnn_epochs = dnn_epochs
        self.dnn_batch_size = dnn_batch_size
        self.learning_rate = learning_rate              
        self.mse = 0        
        self.model_input_path = model_input_path
        self.model_output_path = model_output_path
        self.dnn_model = None

    def init(self, data, bad_cols, target):        
        X, y = get_xy(data, bad_cols, target)
        self.dnn_model = NeuralNetworkModel(X.shape[1], 1, self.learning_rate, self.model_input_path, self.model_output_path)
        return self.dnn_model        

    def train_model_and_get_mse(self, data, bad_cols, target):

        if (self.dnn_model == None):
          print("El modelo no existe, debe obtenerse primero.")  
          return 0       
        else:
          X, y = get_xy(data, bad_cols, target)
          group = data['GameRulesetName']        
          cv = GroupKFold(n_splits=self.n_splits)     

          k = 0
          self.mse = 0
          for train_index, valid_index in cv.split(X, y, group):
              X_train, X_valid = X[train_index], X[valid_index]
              y_train, y_valid = y[train_index], y[valid_index]
              self.dnn_model.fit(X_train, y_train, X_valid, y_valid, int(self.dnn_epochs), self.dnn_batch_size)
              self.mse += mse(y_valid, self.dnn_model.predict(X_valid))
              k += 1

          if (k == 0):
            k = 1
          
          self.mse = self.mse/k       
          
          return self.mse

    def get_mse_after_training(self):        
        return self.mse

    def get_model(self):
        return self.dnn_model

In [12]:
bad_cols = [
    'Id',
    'Behaviour',
    'StateRepetition',
    'Duration',
    'Complexity',
    'BoardCoverage',
    'GameOutcome',
    'StateEvaluation',
    'Clarity',
    'Decisiveness',
    'Drama',
    'MoveEvaluation',
    'StateEvaluationDifference',
    'BoardSitesOccupied',
    'BranchingFactor',
    'DecisionFactor',
    'MoveDistance',
    'PieceNumber',
    'ScoreDifference',
    'EnglishRules',
    'LudRules',
    'num_wins_agent1',
    'num_draws_agent1',
    'num_losses_agent1'
]

target = 'utility_agent1'

Train_data = pl.read_csv(configuracion_parametros.train_path, batch_size = configuracion_parametros.batch_size)
validacion = modelo_validacion(configuracion_parametros.n_splits, configuracion_parametros.dnn_epochs, 
                               configuracion_parametros.dnn_batch_size, configuracion_parametros.learning_rate,
                         configuracion_parametros.model_input_path, configuracion_parametros.model_output_path)
dnn_model = validacion.init(Train_data, bad_cols, target)

X, y = get_xy(Train_data, bad_cols, target)
from sklearn.model_selection import train_test_split
X_ent, X_val, y_ent, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = dnn_model.predict(X_val)
rmse = np.sqrt(mse(y_val, y_pred))
rmse

Modelo cargado exitosamente desde la cuenta:  /kaggle/input/neural-network-with-nash-equilibrium/keras/default/1/saved_model.keras


0.45103908656574937

In [13]:
bad_cols_test = [
    'Id',
    'Behaviour',
    'StateRepetition',
    'Duration',
    'Complexity',
    'BoardCoverage',
    'GameOutcome',
    'StateEvaluation',
    'Clarity',
    'Decisiveness',
    'Drama',
    'MoveEvaluation',
    'StateEvaluationDifference',
    'BoardSitesOccupied',
    'BranchingFactor',
    'DecisionFactor',
    'MoveDistance',
    'PieceNumber',
    'ScoreDifference',
    'EnglishRules',
    'LudRules'    
]

# Función de predicción para la evaluación
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    try: 
        X_test = get_x(test, bad_cols_test)      
        
        # Devolver el resultado de la predicción
        return sample_sub.with_columns(pl.DataFrame({'utility_agent1': dnn_model.predict(X_test)}))
    
    except Exception as e:
        print(f"Error en la predicción: {e}")
        return sample_sub.with_columns(pl.lit(0.0).alias('utility_agent1'))


test = pl.read_csv(configuracion_parametros.test_path)
submission = pl.read_csv(configuracion_parametros.sample_submission_path)

ret = predict(test, submission)
ret

Id,utility_agent1
i64,f32
233234,0.017508
233235,-0.013591
233236,0.01564


In [14]:
# Configuración del servidor de inferencia
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )